In [2]:
import cv2

# Getting Image Pixel coordinates using Mouse

In [3]:
def click_event(event, x, y, flags, params): 
  
    # checking for left mouse clicks 
    if event == cv2.EVENT_LBUTTONDOWN: 
  
        # displaying the coordinates 
        # on the Shell 
        print(x, ' ', y) 
  
        # displaying the coordinates 
        # on the image window 
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font, 
                    1, (255, 0, 0), 2) 
        cv2.imshow('image', img) 
  
    # checking for right mouse clicks      
    if event==cv2.EVENT_RBUTTONDOWN: 
  
        # displaying the coordinates 
        # on the Shell 
        print(x, ' ', y) 
  
        # displaying the coordinates 
        # on the image window 
        font = cv2.FONT_HERSHEY_SIMPLEX 
        b = img[y, x, 0] 
        g = img[y, x, 1] 
        r = img[y, x, 2] 
        cv2.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r), 
                    (x,y), font, 1, 
                    (255, 255, 0), 2) 
        cv2.imshow('image', img) 

In [4]:
# reading image pisel coordinates
img = cv2.imread(r'C:\Users\Piyush\Desktop\cat.jpg', 1) 
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

203   198


In [5]:
import pandas as pd
import numpy as np

# Known Parameters - Camera Matrix, World GCP coordinates, Image GCP Coordinates, Distortion Coefficients

In [28]:
gcp_world = np.array([(0.0, 0.0, 0.0),(1.775, 0.0, 0.0),(1.775, 4.620, 0.0),(0.0, 4.620, 0.0)])
print("World Coordinated of GCps")
print(gcp_world)
# Image coordinates of GCPs - using opencv python
gcp_img = np.array([(275, 204),(331, 204),(331, 308),(275, 308)],dtype="double")
print("Image coordinate of GCPs in pixel\image coordinate")
print(gcp_img)
#Camera Matrix   - from zhangs method
K = np.array([[517.0036881709533, 0, 320],[0, 517.0036881709533, 212],[0, 0, 1]])
print("Camera Matrix")
print(K)
# distortion coefficients
diss_coef = np.array([[0.1128663679798094],[-1.487790079922432],[0],[0],[2.300571896761067]])
#diss_coef = np.zeros((5,1))
print("Distortion Coefficients")
print(diss_coef)

World Coordinated of GCps
[[0.    0.    0.   ]
 [1.775 0.    0.   ]
 [1.775 4.62  0.   ]
 [0.    4.62  0.   ]]
Image coordinate of GCPs in pixel\image coordinate
[[275. 204.]
 [331. 204.]
 [331. 308.]
 [275. 308.]]
Camera Matrix
[[517.00368817   0.         320.        ]
 [  0.         517.00368817 212.        ]
 [  0.           0.           1.        ]]
Distortion Coefficients
[[ 0.11286637]
 [-1.48779008]
 [ 0.        ]
 [ 0.        ]
 [ 2.3005719 ]]


# Finding Homography

In [11]:
def findHomography(world3d,image2d,K,dist):
    (success, rotation_vector, translation_vector) = cv2.solvePnP(world3d,image2d, K, dist, flags=cv2.SOLVEPNP_ITERATIVE)
    print("rotation_vector")
    print(rotation_vector)
    print("translation_vector")
    translation_vector
    print(translation_vector)
    rotation_matrix, jacobian = cv2.Rodrigues(rotation_vector)
    print("Rotation Matrix")
    print(rotation_matrix)
    extrinsic_matrix = cv2.hconcat((rotation_matrix, translation_vector))
    print("Extrinisic parameter matrix")
    print(extrinsic_matrix)
    projection_matrix = np.dot(K,extrinsic_matrix)
    print("Projection Matrix")
    print(projection_matrix)
    homography_matrix = np.delete(projection_matrix,2,axis = 1)
    print("homography_matrix")
    print(homography_matrix)
    return homography_matrix

In [12]:
homography_matrix = findHomography()

World Coordinated of GCps
[[0.    0.    0.   ]
 [1.775 0.    0.   ]
 [1.775 4.62  0.   ]
 [0.    4.62  0.   ]]
Image coordinate of GCPs in pixel\image coordinate
[[275. 204.]
 [331. 204.]
 [331. 308.]
 [275. 308.]]
Camera Matrix
[[517.00368817   0.         320.        ]
 [  0.         517.00368817 212.        ]
 [  0.           0.           1.        ]]
Distortion Coefficeints
[[ 0.11286637]
 [-1.48779008]
 [ 0.        ]
 [ 0.        ]
 [ 2.3005719 ]]
rotation_vector
[[ 0.66147647]
 [-0.0279446 ]
 [ 0.01206996]]
translation_vector
[[-1.39449535]
 [-0.24541537]
 [15.47126946]]
Rotation Matrix
[[ 0.99955339 -0.02011656 -0.02209848]
 [ 0.0022975   0.78903231 -0.61434741]
 [ 0.02979497  0.61402226  0.78872614]]
Extrinisic parameter matrix
[[ 9.99553391e-01 -2.01165645e-02 -2.20984806e-02 -1.39449535e+00]
 [ 2.29750116e-03  7.89032309e-01 -6.14347407e-01 -2.45415372e-01]
 [ 2.97949744e-02  6.14022262e-01  7.88726138e-01  1.54712695e+01]]
Projection Matrix
[[ 5.26307181e+02  1.86086786e+02  

# Finding 3d coordinate of a given 2d image coordinate

In [53]:
def get3d(homography_matrix,image2d):
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    #print("inverse_homography_matrix ")
    #print(inverse_homography_matrix) 
    world3d = np.dot(inverse_homography_matrix,image2d)
    Z = world3d[2,0]
    #print("Z",  Z)
    world3d = world3d/Z
    return world3d

In [39]:
gcp_img   # image pixel coordinates

array([[275., 204.],
       [331., 204.],
       [331., 308.],
       [275., 308.]])

In [44]:
# converting to homogenous coordinates
a = gcp_img
b = np.ones((4,3))
b[:,:-1] = a
b

array([[275., 204.,   1.],
       [331., 204.,   1.],
       [331., 308.,   1.],
       [275., 308.,   1.]])

In [54]:
N = 4
tot3d = []
for i in range(N):
    image2d = b[i]
    image2d = image2d.reshape(-1,1)
    tot3d.append(get3d(homography_matrix,image2d))
tot3d  # 3d coordinates calculated from homography

[array([[0.04752842],
        [0.00737011],
        [1.        ]]),
 array([[1.72558571e+00],
        [1.57324836e-03],
        [1.00000000e+00]]),
 array([[1.87927505],
        [4.62844757],
        [1.        ]]),
 array([[-0.10583623],
        [ 4.61893402],
        [ 1.        ]])]

In [55]:
gcp_world   # actual 3d coordinates

array([[0.   , 0.   , 0.   ],
       [1.775, 0.   , 0.   ],
       [1.775, 4.62 , 0.   ],
       [0.   , 4.62 , 0.   ]])